In [1]:
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df_guj = pd.read_csv('/content/gdrive/MyDrive/HASOC23/guj_train_translated_combined.csv', delimiter = ',')
df_guj

,tweet_id,created_at,text,user_screen_name,label,translated,combined
0,guj001,2023-05-19 15:44:55+00:00,@username @username ઇ વાયડાઈ ના તું હાઈસ ડોબો ...,Red__chilli_,HOF,@username @username e waydi na thou hes dobo\n...,@username @username ઇ વાયડાઈ ના તું હાઈસ ડોબો ...
1,guj002,2023-05-27 16:44:40+00:00,@username @username આવા હલકા કેટલા છે કાય ખબર ...,sureshn25559831,HOF,@username @username does not know how much is ...,@username @username આવા હલકા કેટલા છે કાય ખબર ...
2,guj003,2023-05-25 03:56:05+00:00,@username @username તારી જેમ અમે જ્ઞાતિ માં બટ...,chattansinh,HOF,@username @username like you are not in the ca...,@username @username તારી જેમ અમે જ્ઞાતિ માં બટ...
3,guj004,2023-03-25 08:05:22+00:00,@username આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈...,imkpbhatt,NOT,@username is also a fucking people.\n\n@userna...,@username આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈...
4,guj005,2023-05-24 07:01:42+00:00,"@username @username બેશક, હું તમને મૂર્ખ કહી શ...",irshadvihari,NOT,"@username @username Of course, I can call you ...","@username @username બેશક, હું તમને મૂર્ખ કહી શ..."
...,...,...,...,...,...,...,...
195,guj196,2023-03-25 16:10:38+00:00,@username આજ વિદેશી પ્રિન્ટ મીડિયા માં રાહુલ ગ...,nitinchavda548,NOT,@username has written a lot of Rahul Gandhi in...,@username આજ વિદેશી પ્રિન્ટ મીડિયા માં રાહુલ ગ...
196,guj197,2023-05-26 12:27:01+00:00,@username @username @username ભડવો નયન ચાદર મો...,chattansinh,HOF,@username @username @username,@username @username @username ભડવો નયન ચાદર મો...
197,guj198,2023-05-27 07:26:02+00:00,@username ચોદીના તારા જેવા ને પાકિસ્તાન ભેગી ન...,Rana_Naidu_,HOF,@username Chodina Tara and Pakistan Gathering ...,@username ચોદીના તારા જેવા ને પાકિસ્તાન ભેગી ન...
198,guj199,2023-03-26 09:32:06+00:00,@username @username @username @username @usern...,kkpatel852,NOT,@username @username @username @username @usern...,@username @username @username @username @usern...


# New Section

# New Section

In [4]:
import torch
from transformers import BertModel, BertTokenizerFast


tokenizer = BertTokenizerFast.from_pretrained("setu4993/LaBSE")
if tokenizer.pad_token is None:
  print('adding token')
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#bertmodel = BertModel.from_pretrained("setu4993/LaBSE")
#bertmodel.resize_token_embeddings(len(tokenizer))
#bertmodel.eval()

In [5]:
from transformers import BertModel
from torch import nn
class CustomBERTModel(nn.Module):
    def __init__(self, model):
          super(CustomBERTModel, self).__init__()
          self.bert = model
          ### New layers:

          self.linear1 = nn.Linear(768, 512)
          self.dropout = nn.Dropout(0.3)
          self.relu = nn.LeakyReLU(negative_slope=0.01)
          self.linear2 = nn.Linear(512, 256)


          self.linear3 = nn.Linear(256, 128)


          self.linear4 = nn.Linear(128, 1) ## 2 classes in this example
          self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
          output = self.bert(
               input_ids,
               attention_mask=attention_mask)
          #print(output)
          pooler_output = output['pooler_output']
          #print(sequence_output.shape())
          # sequence_output has the following shape: (batch_size, sequence_length, 768)
          linear1_output = self.linear1(pooler_output)
          relu1_output = self.relu(linear1_output)
          dropout1_output = self.dropout(relu1_output)

          linear2_output = self.linear2(dropout1_output)
          relu2_output = self.relu(linear2_output)


          linear3_output = self.linear3(relu2_output)
          relu3_output = self.relu(linear3_output)

          #sigmoid_output = self.sigmoid(linear2_output)

          linear4_output = self.linear4(relu3_output)
          sigmoid_output = self.sigmoid(linear4_output)
          return sigmoid_output

In [6]:
from transformers import BertModel
from torch import nn
class CustomBERTModel(nn.Module):
    def __init__(self, model):
          super(CustomBERTModel, self).__init__()
          self.bert = model
          ### New layers:
          self.dropout = nn.Dropout(0.3)
          self.linear1 = nn.Linear(768, 256)
          self.relu = nn.LeakyReLU(negative_slope=0.01)
          self.linear2 = nn.Linear(256, 1) ## 2 is the number of classes in this example
          self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
          output = self.bert(
               input_ids,
               attention_mask=attention_mask)
          #print(output)
          pooler_output = self.dropout(output['pooler_output'])
          #print(sequence_output.shape())
          # sequence_output has the following shape: (batch_size, sequence_length, 768)
          linear1_output = self.linear1(pooler_output) ## extract the 1st token's embeddings
          relu_output = self.relu(linear1_output)
          linear2_output = self.linear2(relu_output)
          sigmoid_output = self.sigmoid(linear2_output)

          return sigmoid_output

In [7]:
import pandas as pd
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
class sinhalese_data(Dataset):
  def __init__(self, df_sinh, tokenizer):
    self.dataset = df_sinh
    self.data = list(self.dataset.combined)
    self.labels = list(self.dataset.label)
    self.tokenizer = tokenizer
    #model = BertModel.from_pretrained("setu4993/LaBSE")

  def __len__(self):
    return len(self.data)
  def __getitem__(self, index):
    example = self.data[index]
    inputs = tokenizer(example, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
    input_ids = inputs['input_ids'].squeeze()
    attention_mask = inputs['attention_mask'].squeeze()
    #print(self.labels[index])
    labels = self.labels[index]
    return input_ids, attention_mask, labels

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [9]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_guj, test_size=0.15, stratify = df_guj['label'], random_state=42)

In [10]:
df_test

,tweet_id,created_at,text,user_screen_name,label,translated,combined
164,guj165,2023-05-24 15:35:30+00:00,તેલના ડબ્બા નાં ભાવ ઘટવાનું શરૂ થયુ છે\n\nકોંગ...,RWGujarat,HOF,The price of oil bins has begun to fall\n\nIf ...,તેલના ડબ્બા નાં ભાવ ઘટવાનું શરૂ થયુ છે\n\nકોંગ...
71,guj072,2023-05-26 08:52:15+00:00,@username @username Gujarat નું અપમાન તો કોંગ્...,raj869180,NOT,@username @username Gujarat insults Congress e...,@username @username Gujarat નું અપમાન તો કોંગ્...
2,guj003,2023-05-25 03:56:05+00:00,@username @username તારી જેમ અમે જ્ઞાતિ માં બટ...,chattansinh,HOF,@username @username like you are not in the ca...,@username @username તારી જેમ અમે જ્ઞાતિ માં બટ...
72,guj073,2023-05-26 09:45:01+00:00,@username @username @username 9 વર્ષ મા ઉદ્યોગ...,AshokBabariya8,NOT,@username @username @username businessmen in 9...,@username @username @username 9 વર્ષ મા ઉદ્યોગ...
14,guj015,2023-05-24 07:03:09+00:00,ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર તાલુકામ...,PinalPa10542589,NOT,In Jambusar taluka in Bharuch district in Guja...,ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર તાલુકામ...
148,guj149,2023-05-19 05:20:55+00:00,@username વિરાટ હિન્દુ ને ગઈ રાત નોં ઉજાગરો સે...,Are_O_Sambha,HOF,@username Virat Hindu to light last night ...,@username વિરાટ હિન્દુ ને ગઈ રાત નોં ઉજાગરો સે...
83,guj084,2023-05-28 06:30:00+00:00,પારકી પંચાત' - જુઓ સોમ થી શનિ બપોરે 3 વાગ્યે. ...,Divya_Bhaskar,NOT,Parki Panchayat ' - See Mon to Saturday at 3pm...,પારકી પંચાત' - જુઓ સોમ થી શનિ બપોરે 3 વાગ્યે. ...
12,guj013,2023-05-19 06:46:15+00:00,@username @username આ હું તો બોલીશ વાળા રોનક પ...,Amru1967,NOT,@username @username.\nThe program is not compl...,@username @username આ હું તો બોલીશ વાળા રોનક પ...
69,guj070,2023-05-21 06:33:53+00:00,જુઢો એ કાળિયો કહે મુજ વગર એને કોઈ ના ફાવે.\nગો...,Mutradifalfaz,NOT,The knife does not say no one without me.\nHow...,જુઢો એ કાળિયો કહે મુજ વગર એને કોઈ ના ફાવે.\nગો...
195,guj196,2023-03-25 16:10:38+00:00,@username આજ વિદેશી પ્રિન્ટ મીડિયા માં રાહુલ ગ...,nitinchavda548,NOT,@username has written a lot of Rahul Gandhi in...,@username આજ વિદેશી પ્રિન્ટ મીડિયા માં રાહુલ ગ...


In [11]:
model = torch.load('/content/gdrive/MyDrive/saved_models/last-LA-BSE-Sigmoid-deep-translated-5ep-guj',map_location=torch.device('cpu'))
model.eval()
model.to(device)

CustomBERTModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(501153, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [12]:
dataset = sinhalese_data(df_test, tokenizer)

In [13]:
len(df_test[df_test['label']==1])

0

In [14]:
import torch
from torch.nn import functional as F

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.BCELoss()
batch_size = 4
num_epochs = 10
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [15]:
#inputs = tokenizer(example, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
'''total_loss = 0
criterion = nn.BCELoss()
dataset_test = sinhalese_data(df_test, tokenizer)
data_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
for batch in data_loader:
        # Move batch tensors to the device
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels_lst = batch[2].to(device)
        labels = labels_lst.reshape(len(labels_lst),1)
        labels = labels.float()
        # Zero the gradients


        # Forward pass
        outputs = model(input_ids=input_ids,attention_mask = attention_mask)
               #print(outputs)
        #outputs = F.sigmoid(outputs)
       #print(outputs.shape, labels.shape)
        #print('Output: ', outputs, '\nLabels: ', labels)
        print('outputs:', outputs)
        print('\n')
        print('label:', labels.reshape(len(labels_lst),1))
        loss = criterion(outputs, labels.reshape(len(labels_lst),1))
        print('loss: ',loss)
        #print(loss)
        # Backward pass and optimization
        #loss.backward()
        #optimizer.step()

        total_loss += loss.item()/len(labels_lst)
        print('loss: ', total_loss)

avg_loss = total_loss / len(data_loader)
print(f"Average Loss: {avg_loss:.4f}")'''
pass

In [16]:
#labels_arr = []

dataset_test = sinhalese_data(df_test, tokenizer)
data_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
count = 0
correct = 0
for batch in data_loader:
        # Move batch tensors to the device
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels_lst = batch[2].to(device)
        labels = labels_lst.reshape(len(labels_lst),1)
        labels = labels.float()
        # Zero the gradients


        # Forward pass
        outputs = model(input_ids=input_ids,attention_mask = attention_mask)
        predictions = outputs.tolist()
        #print(predictions)
        for i in range(len(predictions)):
            predictions[i] = round(predictions[i][0])
        print(predictions)
        labels_lst = labels.tolist()

        for j in range(len(labels_lst)):
          labels_lst[j] = int(labels_lst[j][0])
        print(labels_lst)

        for k in range(len(labels_lst)):
          if labels_lst[k]==predictions[k]:
            correct+=1

          count+=1

        print(f'Performance: {correct}/{count}')


print(f'Accuracy: {(correct/count)*100}%')







AttributeError: ignored